In [125]:
import pandas as pd
import numpy as np
import datetime


In [126]:
t0 = datetime.datetime.now()
data = pd.read_excel ("Test_Pandas.xlsx")
t1 = datetime.datetime.now()
print('Execution time: {}'.format(t1 - t0))
data.head()

Execution time: 0:01:07.121887


,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13
3,1064405,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-30 20:16:07
4,20046620,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, ...",35.0,300,Women's Apparel,0,0,0,2016-05-09 01:14:34


In [ ]:
'''
Itemid  - a unique ID of the product
Shopid - a unique ID of the shop
item_name - product title  
item_description - detailed  product description
item_variation - stores variations of a product (e.g. different colours or sizes, in the format like {variation 1 name: variation 1 price, variation 2 name: variation 2 price})
price - how much does the item sold
stock - how many stocks left 
category - which category does the product belongs to 
cb_option - 1 indicates the product is sold by a cross border shop
is_preferred - 1 indicates the product is sold by a preferred shop
sold_count - how many products have been sold 
item_creation_date - when are the product uploaded by the seller
'''

In [3]:
raw_data = data
#Assign all column names to a variable named “columns”
columns = raw_data.columns
print(columns)

Index(['itemid', 'shopid', 'item_name', 'item_description', 'item_variation',
       'price', 'stock', 'category', 'cb_option', 'is_preferred', 'sold_count',
       'item_creation_date'],
      dtype='object')


In [127]:
#How many unique shops are in the dataset?
nb_shop = raw_data['shopid'].nunique()
print('unique shops are in the dataset=',nb_shop)

#How many unique preferred and cross border shops are in the dataset?
nb_shop_pre_cb = raw_data.loc[(raw_data['cb_option']==1) & (raw_data['is_preferred']==1),:]
nb_shop_pre_cb = nb_shop_pre_cb['shopid'].nunique()
print('unique preferred and cross border shops are in the dataset= ',nb_shop_pre_cb)

#How many products have zero sold count?
nb_product_sold = raw_data.loc[raw_data['sold_count']==0,:]
nb_product_sold.head()
nb_product_sold = nb_product_sold['itemid'].nunique()
print('products have zero sold count=',nb_product_sold)

#How many products were created in the year 2018?
nb_product_2018 = raw_data.loc[(raw_data['item_creation_date'] >= '2018-01-01 00:00:00') & (raw_data['item_creation_date'] < '2019-01-01 00:00:00'),:]
print('Number products were created in the year 2018= ',nb_product_2018['itemid'].nunique())
print('Check min date of column creation date=',nb_product_2018['item_creation_date'].min())
print('Check max date of column creation date=',nb_product_2018['item_creation_date'].max())

unique shops are in the dataset= 7856
unique preferred and cross border shops are in the dataset=  158
products have zero sold count= 438092
Number products were created in the year 2018=  177407
Check min date of column creation date= 2018-01-01 02:01:09
Check max date of column creation date= 2018-04-09 03:01:01


In [128]:
#Top 3 Preferred shops’ shopid that have the largest number of unique products
top_3_pre = raw_data.loc[(raw_data['is_preferred']==1),:]
top_3_pre = top_3_pre.groupby('shopid')['itemid'].nunique()
top_3_pre = pd.DataFrame(top_3_pre)
top_3_pre = top_3_pre.reset_index()
top_3_pre.sort_values(["itemid"], axis=0, ascending=False, inplace=True, na_position ='last')
top_3_pre = top_3_pre.rename(columns = {'itemid':'nb_unique_products'})
print('shopid that have the largest number of unique products:\n',top_3_pre.iloc[0:3,])

#Top 3 Categories that have the largest number of unique cross-border products
top_3_cat_cb = raw_data.loc[(raw_data['cb_option']==1),:]
top_3_cat_cb = top_3_cat_cb.groupby('category')['itemid'].nunique()
top_3_cat_cb = pd.DataFrame(top_3_cat_cb)
top_3_cat_cb = top_3_cat_cb.reset_index()
top_3_cat_cb.sort_values(by=['itemid'], axis=0, ascending = False, inplace = True, na_position = 'last')
top_3_cat_cb = top_3_cat_cb.rename(columns = {'itemid':'nb_unique_products'})
print('Top 3 Categories that have the largest number of unique cross-border products:\n',top_3_cat_cb.iloc[0:3,:])



shopid that have the largest number of unique products:
        shopid  nb_unique_products
481  43384791                2034
156   6072241                1998
365  26974701                1037
Top 3 Categories that have the largest number of unique cross-border products:
             category  nb_unique_products
17   Women's Apparel               80494
12  Mobile & Gadgets               58760
10        Men's Wear               51795


In [129]:
#Find Top 3 shopid with the highest revenue (Assumption: the product price has not been changed.) 
top_3_revenue = raw_data
top_3_revenue['revenue'] = top_3_revenue['price'] * top_3_revenue['sold_count']
top_3_revenue = top_3_revenue.groupby('shopid')['revenue'].sum()
top_3_revenue= pd.DataFrame(top_3_revenue)
top_3_revenue = top_3_revenue.reset_index()
top_3_revenue.sort_values(by = ['revenue'],axis=0, ascending = False, inplace = True, na_position='last')
print('Top 3 shopid with the highest revenue: \n',top_3_revenue.iloc[0:3,])

Top 3 shopid with the highest revenue: 
         shopid    revenue
4842  18796234  177897.79
6042  30769353   63491.39
6027  30628794   39852.00


In [130]:
#Find number of products that have more than 3 variations (do not include products with 3 or fewer variations)
raw_data['item_variation'].keys()

RangeIndex(start=0, stop=464433, step=1)

In [148]:
#Use pandas function to identify duplicated listings within each shop (If listing A and B in shop S have the exactly same product title, product detailed description, and price, both listing A and B are considered as duplicated listings)
#Mark those duplicated listings with True otherwise False and store the marking result in a new column named “is_duplicated” 
shop_duplicate = raw_data[raw_data.duplicated(['shopid','item_name','item_description','price'])]
new_df = raw_data
#print(shop_duplicate.index)
#print(raw_data.index)
#new_df['is_duplicated'] = raw_data[raw_data.index.isin(shop_duplicate.index)]
new_df.loc[raw_data.index.isin(shop_duplicate.index), 'is_duplicated'] = True   
new_df.loc[~raw_data.index.isin(shop_duplicate.index), 'is_duplicated'] = False  
#Find duplicate listings that has less than 2 sold count and store the result in a new excel file named “duplicated_listings.xlsx”



#Find the preferred shop shopid that have the most number of duplicated listings


In [149]:
new_df

,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date,revenue,is_duplicated
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.00,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42,0.00,False
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.00,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37,0.00,False
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.00,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13,0.00,False
3,1064405,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.00,1,Mobile & Gadgets,0,0,0,2015-08-30 20:16:07,0.00,False
4,20046620,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, ...",35.00,300,Women's Apparel,0,0,0,2016-05-09 01:14:34,0.00,False
5,20047128,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NIKE SWOOSH DENIM: 35.0, NIKE SWOOSH BLACK: 3...",35.00,299,Men's Wear,0,0,1,2016-05-09 01:18:21,35.00,False
6,78419076,11272000,Hada Labo Super Hydrating Perfect Gel Moisturi...,Hada Labo Super Hydrating Perfect Gel Moisturi...,"{Hydratg UV: 28.72, Super Hydg: 28.72, Whiteni...",28.72,8,Health & Beauty,0,1,5,2016-10-20 11:17:41,143.60,False
7,252736983,11272000,Hada Labo Super Hydrating Perfect Gel Moisturi...,Hada Labo is a skin care line researched & dev...,{},28.72,8,Health & Beauty,0,1,1,2017-05-09 18:33:30,28.72,False
8,793229430,49082000,Women Lace Bra Bustier Crop Tops Soft Mesh Tri...,Material: Terylene Colors: BalckStyle: Sexy Br...,"{S: 9.98, M: 9.98, L: 9.98, XL: 9.98}",9.98,400,Women's Apparel,1,0,0,2017-12-28 19:55:45,0.00,False
9,809446063,49082000,Women Lace Bra Bustier Crop Tops Soft Mesh Tri...,Material: Terylene Colors: BalckStyle: Sexy Br...,"{S: 9.98, M: 9.98, L: 9.98}",9.98,300,Women's Apparel,1,0,0,2018-01-05 16:08:24,0.00,True
